# SPM12w Parameter Files

## Overview

Most functions in SPM12w are driven according to the parameters specified in their associated parameter file types (p,glm,roi,voi) which exist in the _derivatives/userID/code_ directory. In this way, entire analyses (from preprocessing to group-level stats to ROI analyses) can be regenerated with minimal user input.

## The p file

The __p__ parameter file is for preprocessing functional data. The file contains directory naming information along with various settings for preprocessing (e.g., EPI normalization vs. anatomy-based). Typically, we have only one __p__ file per study/task, even though we may have multiple different __glm__ parameter files.

Below we walk through the various lines of a parameters file.

```octave
% User name
p.username = 'wagner.1174';

% Derivative directory & functional task specifications
p.preproc_name = 'epinorm';
p.task = 'tutorial';
```

In this section, the username is provided along with a label to use in naming the derivative directory. Using these two elements, spm12w will create the following directory _derivatives/username/spm-preproc-epinorm_ where the preprocessed data will be deposited. 

```{note} 
The BIDS-compliant rawdata directory is never altered by spm12w. All preprocessing and analyses are stored in the derivatives directory. In this way, the derivatives directory can be deleted without affecting the bids-compliant dataset. Be careful to backup your code directory before doing this (or keep a copy of your code in the sourcedata/code directory). 
```

The task variable allows spm12w to know what task(s) you want to preprocess. If you want to preprocess more than one task, place these in a cell array (i.e., `p.task = {'tutorial','motor'};`)

```octave
% Preprocessing Routines - 1=yes 0=no
p.unwarp = 1; % Unwarping (correct field inhomogeneties)

p.normalize = ’epi’; % Normalize type ('none','epi','spm12','dartel')
p.smoothing = 8;     % Size of smoothing kernel in FWHM (0 for no smoothing)
```

In the section above we specify the preprocessing routines, beginning with SPM's unwarping function (1 = yes, 0 = no), followed by the type of normalization (none, epi, spm12, or dartel) and smoothing kernel size. 

_Below, we describe each parameter in more detail:_

p.username
: Username used when forming directories for derivative data. This corresponds to the _./derivatives/username_ directory. This allows multiple people to work from the same BIDS dataset by segregating their derivatives by user.

p.preproc-name 
: Name used in creating the path for the preprocessing derivatives.

p.task 
: Specifies which task(s) to run preprocessing on. For more than one task, place the tasks in a cell array of strings (i.e. {'tutorial','motor'}).

p.unwarping
: Specifies whether or not to run SPM's unwarping routine. (1=yes, 0=no).

p.normalize
: Specifies which normalization routine to use. 'epi' uses SPM's epi template. 'spm12' uses SPM's default anatomy-based normalization. 'dartel' is a placeholder for dartel-based normalization that we hope to implement in the future.

p.smoothing 
: Size of the smoothing kernel to use after normalization. 

Less commonly used preprocessing options are available in the _spm12w_defaults.m_ file. 

```{warning}
It is important not to edit this file as it will change the defaults for all users. If there is a default you want to override, copy it to your parameters file on an as-needed basis. If you override a default, spm12w will alert you during preprocessing.
```

The defaults in this fill allow you to disable slice time correction, and realignment, and turn off the QA options (e.g., shuffle check, SNR correction, etc.). In order to apply these, __DO NOT EDIT THE DEFAULTS FILE__ but rather copy the variable from the _spm12w_defaults.m_ file into your study's __P__ file. Make sure to change the structure name (i.e., from _def.(fieldname)_ to _p.(fieldname)_). This will override the associated default only for that preprocessing instance.

```{note}
You will know that this is working as spm12w will announce the override as it begins preprocessing. If you do not see the override warning message, then something has gone amiss.
```

## The glm file

The glm file contains parameters specifying first-level GLMs and their contrasts.

```{note}
Each GLM file specifies a single GLM. If you wish to model your task different, or model a different task collected in the same session, you will need to make a new GLM. It is not uncommon for a single study to have more than one GLM file.
```

### GLM name and task specification

The section below specifies what preprocessing data to use for modeling, the glm name, and the task to be modelled.

```octave
% User name
glm.username = 'wagner.1174';

% Directory & functional task specifications
glm.preproc_name = 'epinorm';
glm.glm_name = 'glm_tutorial';
glm.task = 'tutorial';
```

Like in the preprocessing file (i.e., the __p__ file), we begin by specifying the user. In this manner all derivatives (i.e., preprocessing , glm models, etc.) are seperated by user. In the following section, we specify the name of the preprocessing corresponding to the __p.preproc_name__  in the __p__ file).

```{tip}
If you have more than one preprocessing for a given study (e.g., with and without smoothing), you can create a GLM for each preprocessing and specify the name of the preprocessing you'd like to model in the __glm.preproc_name__.
```

Next, we specify the name of the glm which will be used to construct the derivative directory where the results will be stored. Finally, we specify the task to model. Unlike preprocessing, you can only model one task per glm file. 

Unlike in past versions of spm12w, we no longer need to specify an onset directory as all the appropriate task onset parameters are pulled from the associated BIDS task _events.tsv_ file. 

```octave
% GLM conditions corresponding to trial_type column in the events.tsv
glm.events     = {'human','animal','vegetable','mineral'};
glm.blocks     = {};
glm.regressors = {};
```

In this section, we specify whether we wish to model events, blocks or regessors. For standard event-related designs, specify the BIDS __trial_type__ labels inside the cell array, these should match the entries in the associated BIDS _events.tsv_ file. The same is also true of Block designs. 

```{note}
It is possible to specify more than one type of event to model, for instance state-item designs would contain botch events and blocks, whereas PPI models would contain both events and regressors.
```

### GLM contrasts specification

```octave
% GLM Contrasts - Numeric contrasts should sum to zero unless vs. baseline
%               - String contrasts must match condition names.
%               - String contrast direction determined by the placement of 'vs.'
%               - Special keywords: 'housewine' | 'fir_bins' | 'fir_hrf'
% Note that the housewine's allVSbaseline will skip pmods but is not savy
% to complex designs (i.e., where you want all conditions VS baseline but
% not your user supplied regressors under glm.regressors, so in those cases
% you should create your own allVSbaseline and refuse the housewine). 
glm.con.housewine = 'housewine';
glm.con.humVSveg  = [1 0 -1 0]; 
glm.con.aniVSmin  = [0 1 0 -1];
glm.con.minANDhum = 'mineral human'; 
glm.con.humVSall  = 'human vs. animal vegetable mineral';
```

The first contrast in the example above specfied the "housewine" contrast, this is a special keyword that will automatically generate several commonly used contrasts (e.g., all conditions vs. baseline as well as seperate contrasts for each condition vs. baseline). Users should feel free to refuse the housewine if you do not want these contrasts.

The next set of contrasts set numeric and string-based contrasts. spm12w allows for both types of contrasts as well as mixing and matching. For string-based contrasts the labels must match those in the BIDS task-events.tsv __trial_type__ column. In addition, _'vs.'_
is a special keyword that spm12w will use to convert the string contrast into a numeric one. The resulting numeric contrast will be shown in the command window during contrast computing.

```{note}
For numeric contrasts it is not necessary to specify trailing zeros (spm12w will pad the remaining contrast vector with the appropriate zeros).
```

### GLM 2nd-level rfx specification


The final section of the GLM consists of two variables related to the 2nd-level random effects analysis. The first specifies the name of the 2nd-level model and the second describes the contrasts to model. 

```octave
% RFX (2nd-level) specifications
% The assignments below will run seperate random effects one-sample t-tests
% on the conditions allVSbaseline and humVSall.
glm.rfx_name = 'rfx_tutorial';
glm.rfx_conds = {'allVSbaseline','humVSall'};
```

It is also possible to conduct a one-way ANOVA across conditions at the 2nd-level. For this, you need to override the default glm.rfx_type by including the variable below in your GLM file. SPM12w will take care of setting the appropriate subject-level regressors. 

```octave
glm.rfx_type  = 'anova1'; % Normally we don't include type as we usually do 
                          % one-sample and that's the default. Here we have to 
                          % explicitly specify the type.
```

```{tip}
Setting the GLM rfx_type to anova1 will conduct an anove across the conditions specified in the glm.rfx_conds variable. If you want to also conduct regular one-sample t-test across conditions, you will need to specify these in a seperate GLM file. 
```

### Additional GLM model options (default overrides)

As with preprocessing, there are many more options available for GLM estimation. These can be found in the _spm12w_defaults.m_ file. Below is an example of several of these addition options.

```octave
% GLM Model Specifications
def.parametrics = {};    % placeholder for parametrics in case user does not specify
def.runsplit    = 0;     % Seperate GLM per run? Useful for MVPA (%TODO: NOT YET IMPLEMENTED)
def.design_only = 0;     % Design only (i.e., no data)
def.include_run = 'all'; % Specify run to model: 'all' or runs (e.g. [1,3,5])
def.duration    = 0;     % Event/Block Duration in seconds. Will override the durations in the
                         % task event.tsv files. 
def.block_conv  = 1;     % Convolve blocks (0 for state/item) - 1=yes 0=no
def.polort      = 1;     % Order of polynomials: 0:Const|1:Linear|2:Quad|3:Cubic
def.outliers    = 1;     % Include outlier scans as nuissance? (use spm12w_art.m)
def.move        = 1;     % Include motion regressors?
def.trends      = 1;     % Include polynomial trends as nuissance
def.constants   = 1;     % Include n-1 run constants in model for concatenation

% GLM Model Defaults
def.trialtype  = 'trial_type' ; % BIDS column to use for conditions (default: 'trial_type')
def.time       = 'secs';  % Onsets specified in 'scans' or 'secs' Always secs due to bids.
def.hpf        = Inf;     % HPF inf=no cutoff|otherwise cutoff in secs i.e. 128)                               
def.autocorr   = 'none';  % Autocorrelation correction (none | 'AR(1)')                                   
def.demean     = 0;       % Demean condition regressors as in SPM99?
def.orth       = 1;       % Disable w/i trial orth (0) or enable it (1, SPM default) 
def.writeresid = 0;       % Write residual images during GLM modeling (0, SPM default)
def.residname  = 'glm_residuals.nii'; % 4D residuals filename if writeresid = 1

% GLM HRF Specifications:
% OPTIONS:'hrf'
%         'hrf (with time derivative)'
%         'hrf (with time and dispersion derivatives)'
%         'Fourier set'
%         'Fourier set (Hanning)'
%         'Gamma functions'
%         'Finite Impulse Response'
def.hrf       = 'hrf';  %Set to Finite Impulse Response for MIXED designs.
def.hrfwindow = 20;     %For FIR: Length of HRF window 
def.hrfbasis  = 8;      %For FIR: Number of bins per window

% GLM microtime speifications: 
def.tbins     = 16; % Time bin resolution for each scan (16, SPM default)
def.tref      = 1;  % Reference time bin for GLM. Usually timebin corresponding 
                    % to the ref slice during slicetime correction. For Philips
                    % interleaved the first slice acquired is slice 1 so the
                    % the reference timebin is also 1 if p.refslice=1. 

```

glm.parametrics
: Specifies the parametric

glm.include_run
: This setting determines which runs (i.e., sessions) will be modeled. Options are 'all' for all sessions. A single number for only one run (e.g., `glm.include_run = 2;`) or a vector of runs (e.g., `glm.include_run = [1,3];`)

glm.polyord





\item[include\_run:] This setting determines which run (i.e., session) will be modeled. Options are 'all' for all sessions. A single number for only one run (e.g., p.include\_run = 2;) or a vector of runs (.e.g, p.include\_run = [1,3];)
\item[p.polyord:] Number of polynomials to detrend with. Typically we use p.polyord = 1 which is equivalent to a linear trend + constant (i.e., our old linear trend regressor in prior versions of SPM12). However it is possible to include more polynomials, which may be useful for detrending longer runs.
\item[p.move:] Enables or disables inclusion of motion regressors. Under cases of very small motion, it may be advisable to not model motion at all.
\item[p.outliers:] Enables inclusion of dummy regressors for outlier volumes as calculated by art-\_detect (run using SPM12\_art.m). There is no harm to leaving this on, as if no files detailing which scans are outliers are found, then GLM computation will proceed. In this way it's possibly to only model outliers for subjects who have files describing which scans are outliers.
\item[p.demean:] Optional setting to demean task regressors prior to GLM computation. This was the default behavior in SPM99 but, as far as I can tell, was accidentally left out of SPM2 and never repaired as it has no effect on beta estimates. Will Penny from the spm list: \textit{"SPM2 doesn’t mean centre regressors whereas SPM99 did. It is an inconsistency but not one of great import as it will have no effect on the parameter estimates or resulting inference (it will change the estimate of the constant term but nothing else)."} It does, however, result in SPM orthogonality checks which are difficult to interpret. Turning this on should have no effect on results, but make SPM's default orthogonality check easier to interpret.
\item[p.durtime:] Durations are expressed in seconds or TRs. If durations are in seconds (e.g. reaction times), SPM12 will convert them to TRs. This works for block and event-related designs but if you use this all durations must be in seconds (e.g. both the states and items in a state-item design). SPM12 will always warn you during estimation if it divides by TR.
\item[p.disable\_orth:] SPM's within trial type orthogonalization is disabled. SPM defaults to orthogonalizing within trial regressors (i.e., successive parametric regressors or successive FIR regressors). This option allows you to disable this behavior. At present it strikes us as undesirable to orthogonalize FIR regressors. Importantly, this variable expects that you have replaced the stock spm\_fMRI\_design.m file with the modified spm\_fMRI\_design.m file that can be found in SPM12's ADDITIONAL\_FILES directory. 
\end{description}


%\lstinputlisting[language=Matlab, firstline=73, lastline=80]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/P_H8TJAZZ.m}
The following section details some of the advanced settings for HRF modeling, filtering and realignment. 

\begin{description}
\item[p.hrf:] Specifies which model of the HRF to use. 
\item[p.hrfwindow:] For FIR designs: Duration (window) of modeled HRF. Needs to be approximately 20 seconds to capture the full HRF. 
\item[p.hrfbasis:] For FIR designs: Number of basis sets with which to model the HRF. Should be an integer (preferably p.hrfwindow / TR). So for a 20 second window, with a TR of 2.5, p.hrfbasis would be 8 (i.e. 8 time bins of 2.5s). 
\item[p.hpf and p.autocorr:] Additional options include high pass filtering and serial autocorrelation correction. These should only be used if you have a single run of data and are feeling adventurous. Although, according to Darren Gitelmen at an MGH workshop, it is not quite a cardinal sin to run HPF and autorcorrelation correction on concatenated runs, but nevertheless this has not been our convention. 
\item[p.refslice and p.realign\_rtm:] Specifies which slice to use as reference slice for slicetime correction and whether realignment should take place relative to the 1st slice (1pass) or if realignment should use a 2pass solution realigning to the 1st run then to the session mean. In general we always use slice 1, but this can vary depending on acquisition sequence.
\item[p.overridedel:] If set to 0, SPM12 will always ask if you wish to delete prior analyses before overwriting them (i.e., when redoing preprocessing or GLM computation). The reasoning for this setting is to ensure that analysis directories remain free of clutter from prior analyses, SPM12 will typically delete the entire FUNCTIONAL directory (saving only outlier files) and check out fresh copies from the NIFTI directory. If you decide to store your diary and personal photographs inside a subject's FUNCTIONAL or ANATOMICAL directory, you will lose them. p.overridedel = 0 will cause SPM12 to ask you for confirmation prior to deleting prior data directories. Although, this may seem desirable it requires you to answer this question for every subject. 
\end{description}

%\lstinputlisting[language=Matlab, firstline=82, lastline=100]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/P_H8TJAZZ.m}
There are some additionally settings in the P file no discussed here. To find out more, read the comments in the P files provided with the example dataset. 

\subsection{The CON file}
The study contrasts file specifies the type of contrast to perform on the conditions from the GLM. Contrasts are zero padded to fill in nuisance regressors. This is especially useful in situations where the number of regressors in the model differs per subject (e.g. if p.outliers = 1) or for mixed designs and FIR models where the number of regressors can easily reach 100 (see CON\_H8TJAZZ\_MIXED.m for an example of some tricks for making organized contrast vectors for a state-item study). The way this works is that SPM12 will searche through the regressor names and count the number of 'r' regressors (which is the name we reserve for nuisance regressors). It is therefore important to keep in mind that this 'trick' will fail if you hack in any custom nuisance regressors and don’t name them 'r'. If you let SPM12 make the nuisance regressors for you, then this should always be correct. 

Protip: ensure that your contrast vectors always sum to zero (except for vs. baseline) and that your contrast weights never exceed one. For instance, [3 -1 -1 -1] while a valid contrast that will generate an appropriate statistical map, the underlying beta values are multiplied by the contrast weights. Thus, when you go to extract con values for an ROI analysis, the values will be scaled by the weights. All statistics on such values will be valid, but when you go to plot the results, the y-axis may appear to be overly large! 

%\lstinputlisting[language=Matlab, firstline=7, lastline=17]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/CON_H8TJAZZ.m}
%\lstinputlisting[language=Matlab, firstline=19, lastline=28]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/CON_H8TJAZZ.m}

## The roi file
The __roi__ parameters file specifies how to run the ROI analyses using spm12w's roitool (i.e., `spm12w_roitool.m`). This file is used to determine what type (image mask or spherical) of ROI to generate, what model to grab the parameter estiamtes from and what statistics to conduct (e.g., descriptives, t-tests, correlations).

The following section of the ROI parameter file specifies where the ROI analysis results will be written (roi\_dir) which RFX directory contains the con images (rfx\_dir). The optional location of the directory containing a priori ROIs (i.e., anatomical or some other source; roi\_img\_dir). And finally the filename of excel spreadsheet files containing roi specifications and subject variables (see the xlsx files provided with the example dataset to understand the layout). Note that the roi\_spec\_file is optional. ROI specifications can also be defined in a variable (see below) and will be used if the roi\_spec\_file does not exist.

%\lstinputlisting[language=Matlab, firstline=7, lastline=13]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/ROI_H8TJAZZ.m}
This section describes the conditions from which the parameter estimates will be extracted. These conditions must match those in the CON parameter file that is associated with this particular RFX analysis. Put differently, SPM12\_roitool.m will extract parameter estimates from files in the RFX/RFX\_H8TJAZZ/humVSbas directory (and so forth). 

%\lstinputlisting[language=Matlab, firstline=16, lastline=23]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/ROI_H8TJAZZ.m}
This section defines the roi specs requiring a region name, an MNI coordinate and a size in mm's (for spherical ROIs). If a filename is provided instead of an ROI size then SPM12\_roitool will search for this file in the roi\_img\_dir specified earlier. Finally, if a roi spec excel file was specified and exists, this r.roi\_specs variable will be ignored.

%\lstinputlisting[language=Matlab, firstline=25, lastline=33]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/ROI_H8TJAZZ.m}
This section describes the statistics that will be performed on the parameter estimates extracted from for each condition. Condition names can be used to construct formulae for analysis (i.e. 'vegVSbas-minVSbas') which SPM12\_roitool will interpret appropriately. If 'all\_conditions' is written instead, then SPM12\_roitool will perform the defined statistic on all conditions in r.conditions. 
For correl and t-test2, SPM12\_roitool requires a subject variables excel file defining group membership and, for each subject, a value for one or more variables to use in correlation analyses.
%\lstinputlisting[language=Matlab, firstline=35, lastline=45]{../EXAMPLE_DATA/2013_H8TJAZZ_SPM12_r5236/SCRIPTS/ROI_H8TJAZZ.m}